In [5]:
rm -rf /kaggle/working/*

In [6]:
!pip install -q -U google-genai

In [7]:
!git clone --branch main https://github.com/giankev/Ancient-to-Modern-Italian-Automatic-Translation.git

Cloning into 'Ancient-to-Modern-Italian-Automatic-Translation'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 47 (delta 20), reused 30 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 52.69 KiB | 4.05 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [8]:
from kaggle_secrets import UserSecretsClient
import huggingface_hub
import os

try:
    user_secrets = UserSecretsClient()
    API_KEY = user_secrets.get_secret("api_google_ai")
    print("API KEY Loaded")
except Exception as e:
    print(f"Error during programmatic login: {e}")
    print("Please ensure your HF_TOKEN secret is correctly set in Kaggle.")

API KEY Loaded


In [9]:
from time import sleep

In [10]:
import pandas as pd

csv_translation = "/kaggle/working/Ancient-to-Modern-Italian-Automatic-Translation/OpusMT-it-translation/opusMT_translations.csv"
df = pd.read_csv(csv_translation)

In [11]:
df.head()

,sentence,Translations
0,quella guerra ben fatta l' opera perché etc. E...,Quella guerra ben fatto il lavoro perché ecc E...
1,"crudele, e di tutte le colpe pigli vendetta, c...","crudele, e di tutti i difetti prendere vendett..."
2,Non d' altra forza d' animo fue ornato Ponzio ...,Nessun'altra forza della mente fu adorna di Po...
3,Se questo piace a tutti e se 'l tempo hae biso...,Se questo piace a tutti e se e' necessario che...
4,Officio di questa arte pare che sia dicere app...,L'ufficio di questa arte sembra essere quello ...


In [12]:
from google import genai

client = genai.Client(api_key=API_KEY)

In [13]:
def prompt_evaluation(original: str, translated: str):
    prompt =  f"""Valuta la seguente traduzione dall'italiano antico all'italiano moderno.
                Concentrati sulla fedeltà semantica e la correttezza grammaticale.
                Fornisci un punteggio da 1 a 5 basandoti sulla seguente rubrica:
                1. Traduzione completamente inaccettabile: la traduzione non ha pertinenza con il significato originale, la frase generata è incomprensibile (gibberish) o senza senso.
                2. Errori semantici gravi, omissioni o aggiunte sostanziali rispetto alla frase originale. Gli errori sono di natura semantica e sintattica. È qualcosa che un umano non scriverebbe mai.
                3. Traduzione parzialmente errata: la traduzione è mediocre, contiene errori, ma sono per lo più errori minori, come refusi o piccoli errori semantici.
                4. Buona traduzione: la traduzione è per lo più corretta, sostanzialmente fedele al testo originale, ma lo stile potrebbe non corrispondere perfettamente alla frase originale; è comunque fluente, comprensibile e semanticamente accettabile.
                5. Traduzione perfetta: la traduzione è accurata, fluente, completa e coerente. Ha mantenuto il significato originale il più possibile.
                fornisci solo il numero del punteggio (da 1 a 5).
                frase originale: "{original}"
                frase tradotta: "{translated}"
                
                """
    return prompt

In [15]:
results = []
for index, row in df.iterrows():
    
    original = row["sentence"]
    translation = row["Translations"]
    prompt = prompt_evaluation(original, translation)

    response = client.models.generate_content(
             model="gemini-2.0-flash-lite",
             contents=prompt
    )

    results.append({
        "sentence": original,
        "translation": translation,
        "score": response.text
    })

    sleep(2) 

In [16]:
df_results = pd.DataFrame(results)
df_results.to_csv("/kaggle/working/evaluation_opusMT.csv", index=False, encoding='utf-8')

In [17]:
import json

with open("/kaggle/working/evaluation_opusMT.jsonl", "w", encoding="utf-8") as f:
    for entry in results:
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')